In [ ]:
import os

os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import bambi as bmb
import arviz as az 
from pysal.lib import weights
az.style.use("arviz-darkgrid")


dr = DataReg()

In [ ]:
df = dr.regular_data(naics="11")
df

In [ ]:
df[df["fereign"] == 0]

In [ ]:
half_norm = bmb.Prior("HalfNormal", mu=0, sd=3)

In [ ]:
model = bmb.Model(
    "log_total_employment ~ 0 + zipcode + date2 + k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security",
    df,
    dropna=True,
)
model

In [ ]:
results = model.fit(
    inference_method="nutpie",
    sample_kwargs={"target_accept": 0.8},
    cores=10,
    chains=10,
    random_seed=787,
)

In [ ]:
t =az.summary(results)
t

In [ ]:
t[t["r_hat"] >= 1.05]

In [ ]:
az.plot_trace(results, compact=False)